In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json

In [2]:
csv_file = "./Resources/USvideos.csv"
youtube_data_df = pd.read_csv(csv_file, usecols=['video_id', 'title', 'views', 'tags'])

youtube_data_df.head()

,video_id,title,tags,views
0,2kyS6SvSYSE,WE WANT TO TALK ABOUT OUR MARRIAGE,SHANtell martin,748374
1,1ZAPwfrtAFY,The Trump Presidency: Last Week Tonight with J...,"last week tonight trump presidency|""last week ...",2418783
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...","racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434
3,puqaWrEC7tY,Nickelback Lyrics: Real or Fake?,"rhett and link|""gmm""|""good mythical morning""|""...",343168
4,d380meD0W0M,I Dare You: GOING BALD!?,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731


In [3]:
json_file = './Resources/trending.json'
with open(json_file, 'r') as j:
     data = json.loads(j.read())

tiktok_data_df = pd.DataFrame(data['collector'], columns=['id', 'text', 'playCount', 'hashtags'])
tiktok_data_df.head()

,id,text,playCount,hashtags
0,6907228749016714497,Confidence went 📈,44800,[]
1,6875468410612993286,Quiet Zone... follow me on insta: joeysofo. Co...,838100,[]
2,6898699405898059010,Iphone bend test🤗 #tiktok #viral #fyp #iphone ...,15300000,"[{'id': '23428', 'name': 'tiktok', 'title': ''..."
3,6902819837345533186,,94900,[]
4,6905635666588192002,小技です👟✨#tiktok教室#tutorial,115300,"[{'id': '1627704798586882', 'name': 'tiktok教室'..."


In [4]:
# Connect to Database 
rds_connection_string = "trends_project:password@localhost:5432/trending_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [5]:
engine.table_names() 

<ipython-input-5-8f1edc7208d0>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['trending', 'tags', 'customer_name', 'trending_db']

In [6]:
new_youtube_df = youtube_data_df.rename(columns={"video_id": "platform_id", "views": "view_count"})
new_youtube_df.head()

,platform_id,title,tags,view_count
0,2kyS6SvSYSE,WE WANT TO TALK ABOUT OUR MARRIAGE,SHANtell martin,748374
1,1ZAPwfrtAFY,The Trump Presidency: Last Week Tonight with J...,"last week tonight trump presidency|""last week ...",2418783
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...","racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434
3,puqaWrEC7tY,Nickelback Lyrics: Real or Fake?,"rhett and link|""gmm""|""good mythical morning""|""...",343168
4,d380meD0W0M,I Dare You: GOING BALD!?,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731


In [7]:
new_tiktok_data_df = tiktok_data_df.rename(columns={"id": "platform_id", "text": "title", "hashtags": "tags", "playCount": "view_count"})
new_tiktok_data_df.head()

,platform_id,title,view_count,tags
0,6907228749016714497,Confidence went 📈,44800,[]
1,6875468410612993286,Quiet Zone... follow me on insta: joeysofo. Co...,838100,[]
2,6898699405898059010,Iphone bend test🤗 #tiktok #viral #fyp #iphone ...,15300000,"[{'id': '23428', 'name': 'tiktok', 'title': ''..."
3,6902819837345533186,,94900,[]
4,6905635666588192002,小技です👟✨#tiktok教室#tutorial,115300,"[{'id': '1627704798586882', 'name': 'tiktok教室'..."


In [9]:
new_youtube_df.to_sql('trending', if_exists='replace', con=engine, index=False) 
engine.execute('select * from trending').fetchall()

[('2kyS6SvSYSE', 'WE WANT TO TALK ABOUT OUR MARRIAGE', 'SHANtell martin', 748374),
 ('1ZAPwfrtAFY', 'The Trump Presidency: Last Week Tonight with John Oliver (HBO)', 'last week tonight trump presidency|"last week tonight donald trump"|"john oliver trump"|"donald trump"', 2418783),
 ('5qpjK5DgCt4', 'Racist Superman | Rudy Mancuso, King Bach & Lele Pons', 'racist superman|"rudy"|"mancuso"|"king"|"bach"|"racist"|"superman"|"love"|"rudy mancuso poo bear black white official music video"|"iphone x by pinea ... (17 characters truncated) ... "hannahstocking"|"rudymancuso"|"inanna"|"anwar"|"sarkis"|"shots"|"shotsstudios"|"alesso"|"anitta"|"brazil"|"Getting My Driver\'s License | Lele Pons"', 3191434),
 ('puqaWrEC7tY', 'Nickelback Lyrics: Real or Fake?', 'rhett and link|"gmm"|"good mythical morning"|"rhett and link good mythical morning"|"good mythical morning rhett and link"|"mythical morning"|"Season ... (179 characters truncated) ... ickelback (musical group)"|"rock"|"music"|"lyrics"|"chad k

In [13]:
new_tiktok_data_df.to_sql('trending', if_exists='append', con=engine, index=False) 
engine.execute('select * from trending').fetchall()

ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'dict'
[SQL: INSERT INTO trending (platform_id, title, view_count, tags) VALUES (%(platform_id)s, %(title)s, %(view_count)s, %(tags)s)]
[parameters: ({'platform_id': '6907228749016714497', 'title': 'Confidence went 📈', 'view_count': 44800, 'tags': []}, {'platform_id': '6875468410612993286', 'title': 'Quiet Zone... follow me on insta: joeysofo. Comment where you wanna see me blade next. Reply to @dwight_schnuute', 'view_count': 838100, 'tags': []}, {'platform_id': '6898699405898059010', 'title': 'Iphone bend test🤗 #tiktok #viral #fyp #iphone #test #bend', 'view_count': 15300000, 'tags': [{'id': '23428', 'name': 'tiktok', 'title': '', 'cover': ''}, {'id': '20884', 'name': 'viral', 'title': '', 'cover': ''}, {'id': '229207', 'name': 'fy ... (59 characters truncated) ... one', 'title': '', 'cover': ''}, {'id': '15134', 'name': 'test', 'title': '', 'cover': ''}, {'id': '16962', 'name': 'bend', 'title': '', 'cover': ''}]}, {'plat